# Imports

In [1]:
# Used to pull data from Reddit
import praw
import pandas as pd
import datetime as dt

from gensim.models import Word2Vec
from nltk.corpus import brown
import nltk
import tensorflow as tf
import tensorflow_hub as hub
import multiprocessing
from google.cloud import bigquery

#nltk.download('punkt')
#nltk.download('brown')

In [87]:
sql_query = '''
SELECT 
    r.subreddit,
    r.body
FROM 
  `fh-bigquery.reddit_comments.2019_03` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'repbulicans')
'''

comment_df = pd.read_gbq(sql_query,
                         project_id='w266-240122',
                         dialect='standard')

In [140]:
sql_query = """
SELECT 
    r.subreddit,
    r.title,
    r.selftext,
    r.is_self
FROM 
  `fh-bigquery.reddit_posts.2019_03` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'repbulicans')
"""

post_df = pd.read_gbq(sql_query,
                      project_id='w266-240122',
                      dialect='standard')

# Emebdding from Democratic Leaning Subreddits

### Text PreProcessing

In [113]:
# Subreddit filter for democratic leaning subreddits
dem_subreddit = ['democrats', 'progressive']

# Body text from post dataframe
dem_post_text = list(post_df[(post_df['is_self'] == True) &
                             (~post_df['selftext'].isin(['[removed]', '[deleted]'])) &
                             post_df['subreddit'].isin(dem_subreddit)]['selftext'].unique())

# Add in text from title
dem_post_text += list(post_df[post_df['is_self'] == True]['title'].unique())

# Add in text from post comments
dem_post_text += list(comment_df[(comment_df['subreddit'].isin(dem_subreddit)) & 
                                 (~comment_df['body'].isin(['[removed]', '[deleted]']))]['body'])

# Put all text into dataframe and drop dupes
dem_df = pd.DataFrame(dem_post_text, columns=['text'])
dem_df.drop_duplicates(inplace=True)

# Tokenize at sentence level
dem_df['sent_tokenized'] = dem_df['text'].apply(nltk.sent_tokenize)
dem_df['sent_count'] = dem_df['sent_tokenized'].apply(lambda x: len(x))

# Put tokenized body into a list
dem_sent_list = list(dem_df[dem_df['sent_count'] > 0]['sent_tokenized'].apply(pd.Series).stack().unique())

# Put list into dataframe
ds_df = pd.DataFrame(dem_sent_list, columns=['sentence'])

# Tokenize each sentence at the word level
ds_df['word_token'] = ds_df['sentence'].apply(nltk.word_tokenize)

In [ ]:
# TODO remove symbols, contractions
# TODO lowercase words?
# TODO Treat numbers as something else
# Add beginning and end of sentence?
# TODO how do we deal with stop words?

### Skipgram Embedding

In [115]:
# train word embedding
dem_embedding = Word2Vec(list(ds_df['word_token']), size=100, window=5, min_count=5, negative=15, iter=10, workers=6)

In [139]:
dem_embedding.wv.similar_by_word('Andrew')

[('Buttigieg', 0.8853778839111328),
 ('Populism', 0.8778499364852905),
 ("O'Rourke", 0.876208484172821),
 ('Cory', 0.8709084391593933),
 ('injury', 0.8704582452774048),
 ('Jay', 0.8690897822380066),
 ('Says', 0.8690533638000488),
 ('Elizabeth', 0.8658692836761475),
 ('Stacey', 0.8642362356185913),
 ('Atlantic', 0.8636647462844849)]

In [116]:
dem_embedding.wv.similar_by_word('Trump')

[('trump', 0.7485096454620361),
 ('he', 0.6877127885818481),
 ('president', 0.6715365648269653),
 ('Russia', 0.6475684642791748),
 ('He', 0.6389645338058472),
 ('McConnell', 0.6372545957565308),
 ('Clinton', 0.6367909908294678),
 ('Hillary', 0.6154500246047974),
 ('President', 0.612384557723999),
 ('She', 0.6113145351409912)]

In [123]:
dem_embedding.wv.similar_by_word('McConnell')

[('nobody', 0.7864992022514343),
 ('Cornyn', 0.7798274755477905),
 ('drop', 0.7716587781906128),
 ('necessary', 0.762731671333313),
 ('fun', 0.7519711256027222),
 ('Trumps', 0.7489361763000488),
 ('investigated', 0.7456454038619995),
 ('Apple', 0.7444183826446533),
 ('9/11', 0.7405687570571899),
 ('Ivanka', 0.7376561760902405)]

In [8]:
dem_embedding.wv.similar_by_word('Trump')

[('trump', 0.8643887042999268),
 ('he', 0.7681137323379517),
 ('He', 0.7501710653305054),
 ('Obama', 0.7333434820175171),
 ('Barr', 0.7009626626968384),
 ('Bernie', 0.6909111738204956),
 ('Biden', 0.6875118613243103),
 ('him', 0.6806504726409912),
 ('Russia', 0.6793426871299744),
 ('President', 0.6673457026481628)]

### ELMo

In [20]:
# Download ELMo to a local directory
!mkdir module_elmo

# Download the module, and uncompress it to the destination folder. 
!curl -L "https://tfhub.dev/google/elmo/2?tf-hub-format=compressed" | tar -zxvC module_elmo

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  331M    0   415    0     0    369      0  10d 21h  0:00:01  10d 21h   369x assets/
x saved_model.pb
x tfhub_module.pb
x variables/
x variables/variables.index
100  331M  100  331M    0     0  2054k      0  0:02:45  0:02:45 --:--:-- 2379k 2614kk      0  0:02:59  0:01:26  0:01:33 2755k   0  2087k      0  0:02:42  0:01:43  0:00:59 3369k      0  0:02:41  0:01:51  0:00:50 2300k  0:01:56  0:00:44 2391k0:02:44  0:02:25  0:00:19 2082k



In [53]:
elmo = hub.Module('module_elmo', trainable=True)
embeddings = elmo(['trump'], signature="default", as_dict=True)['elmo']

with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embeddings)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0712 23:35:19.670152 4588623296 saver.py:1483] Saver not created because there are no variables in the graph to restore


# Questions
* Sentiment analysis labeled data
* How would we go about scoring the performance of our models?
    * I proposed to use the scores that are available on the reddit
* Should we train the sentiment classifier at a sentence or post level?
    * Each post may contain multiple sentences


### Data from r/Republican Subreddit

In [91]:
len(list(dem_posts['sent_tokenized_title'].apply(pd.Series).stack()))

1497

In [92]:
1497 + 714

2211

In [90]:
len(test)

2211

In [61]:
republicans = reddit.subreddit('Republican')

In [76]:
posts = []
# for post in democrats.top(limit = 1):
for post in republicans.hot(limit = 1000):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created, post.is_self, post.link_flair_text])
rep_posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created', 'is_self', 'link_flair_text'])
created = rep_posts["created"].apply(lambda x: dt.datetime.fromtimestamp(x))
rep_posts['created'] = created